[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/reznikmm/ada-howto/advent-2020?filepath=%2Fhome%2Fjovyan%2Fnb%2F19%2F19.ipynb)

# Day 19

We need some standard packages to read input and keep data.

In [1]:
with Ada.Text_IO;
with Ada.Containers.Vectors;
with Ada.Containers.Doubly_Linked_Lists;
with Ada.Containers.Hashed_Maps;
with Ada.Strings.Fixed;
with Ada.Strings.Maps.Constants;

Let's use a kind of [Pike virtual machine](https://swtch.com/~rsc/regexp/regexp2.html). It execute 4 kinds of command:
 * Literal - accept given literal
 * Call - process a given rule and return
 * Fork - create another copy of machine and continue execution from alternative command
 * Done - return from current rule
 
We create as many copies of VM as we want. Each of them has a dedicated call stack. All VM work with the same position of the input string and use the same program (a sequence of commands).

In [2]:
type Rule_Id is new Natural;
type Command_Kind is (Literal, Call, Fork, Done);
type Address is new Positive;

type Command (Kind : Command_Kind := Command_Kind'First) is record
   case Kind is
      when Literal =>
         Char : Character;
      when Call =>
         Rule : Rule_Id;
      when Fork =>
         Alternative : Address;
      when Done =>
         null;
   end case;
end record;
       
package Command_Vectors is new Ada.Containers.Vectors (Address, Command);

Program : Command_Vectors.Vector;

Each rule points to a start command. Let's keep rules in a hash table

In [3]:
function Hash (Value : Rule_Id) return Ada.Containers.Hash_Type is
  (Ada.Containers.Hash_Type'Mod (Value));

package Rule_Maps is new Ada.Containers.Hashed_Maps
  (Key_Type        => Rule_Id,
   Element_Type    => Address,
   Hash            => Hash,
   Equivalent_Keys => "=");

Rules : Rule_Maps.Map;

A utulity subprogram to parse `Rule_Id` from a string:

In [4]:
procedure Get_Id
  (Line : String; Value : out Rule_Id; Last : out Positive);
--  Find rule id at the beginning of a Line and shift Last.

procedure Get_Id
  (Line : String; Value : out Rule_Id; Last : out Positive)
is
   First : Positive;
begin
   Ada.Strings.Fixed.Find_Token
     (Source => Line,
      Set    => Ada.Strings.Maps.Constants.Decimal_Digit_Set,
      Test   => Ada.Strings.Inside,
      First  => First,
      Last   => Last);
   Value := Rule_Id'Value (Line (First .. Last));
end Get_Id;

Define input file and a counter

In [5]:
Count : Natural := 0;
Input : Ada.Text_IO.File_Type;

Let's parse rules and create a program on the fly

In [6]:
Ada.Text_IO.Open (Input, Ada.Text_IO.In_File, "/tmp/input");

loop
   declare
      Line : String := Ada.Text_IO.Get_Line (Input);
      Id   : Rule_Id;
      Id_2 : Rule_Id;
      Id_3 : Rule_Id;
      Last : Positive;
   begin
      exit when Line = "";
      Get_Id (Line, Id, Last);
      pragma Assert (Line (Last + 1 .. Last + 2) = ": ");
      Rules.Insert (Id, Program.Last_Index + 1);

      if Line (Last + 3) = '"' then
         Program.Append ((Literal, Line (Last + 4)));  --  X: "a"
         Last := Last + 5;
      else
         Get_Id (Line (Last + 3 .. Line'Last), Id_2, Last);

         if Last = Line'Last then
            Program.Append ((Call, Id_2));  --  X: N
         elsif Line (Last + 2) = '|' then  --  X: N | M
            Program.Append ((Fork, Program.Last_Index + 4));
            Program.Append ((Call, Id_2));
            Program.Append ((Kind => Done));
         else  --  X: N M...
            Get_Id (Line (Last + 1 .. Line'Last), Id_3, Last);

            if Last = Line'Last then  --  X: N M
               Program.Append ((Call, Id_2));
               Program.Append ((Call, Id_3));
            elsif Line (Last + 2) = '|' then  --  X: N M | ...
               Program.Append ((Fork, Program.Last_Index + 5));
               Program.Append ((Call, Id_2));
               Program.Append ((Call, Id_3));
               Program.Append ((Kind => Done));
            else  --  X: N M K
               Program.Append ((Call, Id_2));
               Program.Append ((Call, Id_3));
               Get_Id (Line (Last + 1 .. Line'Last), Id_3, Last);
               Program.Append ((Call, Id_3));
            end if;
         end if;

         if Last < Line'Last then  --  X: ... | N ..
            pragma Assert (Line (Last + 2) = '|');
            Get_Id (Line (Last + 3 .. Line'Last), Id_2, Last);
            Program.Append ((Call, Id_2));

            if Last < Line'Last then  --  X: ... | N M
               Get_Id (Line (Last + 1 .. Line'Last), Id_3, Last);
               Program.Append ((Call, Id_3));
            end if;
         end if;
      end if;
      pragma Assert (Last = Line'Last);

      Program.Append ((Kind => Done));
   end;
end loop;

Now define a `Match` function. It will execute the program in a number of treads. Each thread runs until it read the current character. If the character match then the tread saved to execute in a next round for next character.

In [7]:
function Match (Line : String) return Boolean;
--  Check is the Line match

function Match (Line : String) return Boolean is
   package Address_Lists is new Ada.Containers.Doubly_Linked_Lists (Address);

   type Thread is record
      Stack : Address_Lists.List;
   end record;

   package Thread_Lists is new Ada.Containers.Doubly_Linked_Lists (Thread);

   Init : Address_Lists.List;
   Prev : Thread_Lists.List;
begin
   Init.Append (Rules (0));  --  Start execution from rule 0
   Prev.Append ((Stack => Init));

   for Char of Line loop  --  Iterate over each character
      declare
         Next  : Thread_Lists.List;  --  Set of thread to run in the next iteration
      begin
         while not Prev.Is_Empty loop  --  While we have a thread to execute
            declare
               Active : Thread := Prev.Last_Element;
               Addr   : Address := Active.Stack.Last_Element;
               Cmd    : Command := Program (Addr);  --  Next command to execute
            begin
               Prev.Delete_Last;  --  Delete current thread from the list
               Active.Stack.Delete_Last;  --  Delete last stack frame

               case Cmd.Kind is
                  when Literal =>
                     if Cmd.Char = Char then
                        Next.Append (Active);  --  Keep thread to next round
                     end if;
                  when Call =>
                     Active.Stack.Append (Addr + 1);
                     Active.Stack.Append (Rules (Cmd.Rule));
                     Prev.Append (Active);  --  Schedule thread again
                  when Fork =>
                     Active.Stack.Append (Addr + 1);
                     Prev.Append (Active);  --  Schedule thread again
                     Active.Stack.Delete_Last;
                     Active.Stack.Append (Cmd.Alternative);
                     Prev.Append (Active);  --  Schedule alternative thread
                  when Done =>
                     if not Active.Stack.Is_Empty then
                        Prev.Append (Active);  --  Schedule thread again
                     end if;
               end case;
            end;
         end loop;

         Prev.Move (Source => Next);  --  Swap next thread set
      end;
   end loop;

   --  Find a completed thread
   while not Prev.Is_Empty loop  --  While we have a thread to check
      declare
         Active : Thread := Prev.Last_Element;
         Addr   : Address := Active.Stack.Last_Element;
         Cmd    : Command := Program (Addr);
      begin
         Prev.Delete_Last;
         Active.Stack.Delete_Last;

         case Cmd.Kind is
            when Literal =>
               null;
            when Call =>
               Active.Stack.Append (Addr + 1);
               Active.Stack.Append (Rules (Cmd.Rule));
               Prev.Append (Active);
            when Fork =>
               Active.Stack.Append (Addr + 1);
               Prev.Append (Active);
               Active.Stack.Delete_Last;
               Active.Stack.Append (Cmd.Alternative);
               Prev.Append (Active);
            when Done =>
               if Active.Stack.Is_Empty then
                  return True;  --  Thread is complete!
               else
                  Prev.Append (Active);
               end if;
         end case;
      end;
   end loop;

   return False;
end Match;

Now we read lines and count matched

In [8]:
while not Ada.Text_IO.End_Of_File (Input) loop
   declare
      Line : String := Ada.Text_IO.Get_Line (Input);
   begin
      Count := Count + Boolean'Pos (Match (Line));
   end;
end loop;

Ada.Text_IO.Put_Line (Count'Image);

 144


To solve Path II just modify two rules:

In [9]:
--  8: 42 | 42 8

Rules.Include (8, Program.Last_Index + 1);
Program.Append ((Fork, Program.Last_Index + 4));
Program.Append ((Call, 42));
Program.Append ((Kind => Done));
Program.Append ((Call, 42));
Program.Append ((Call, 8));
Program.Append ((Kind => Done));

--  11: 42 31 | 42 11 31
Rules.Include (11, Program.Last_Index + 1);
Program.Append ((Fork, Program.Last_Index + 5));
Program.Append ((Call, 42));
Program.Append ((Call, 31));
Program.Append ((Kind => Done));
Program.Append ((Call, 42));
Program.Append ((Call, 11));
Program.Append ((Call, 31));
Program.Append ((Kind => Done));

Reopen the `Input` and skip rules

In [10]:
Ada.Text_IO.Close (Input);
Count := 0;
Ada.Text_IO.Open (Input, Ada.Text_IO.In_File, "/home/jovyan/md/19/input");
--  Skip rules because we already have them
loop
   exit when Ada.Text_IO.Get_Line (Input) = "";
end loop;

Count matches again:

In [11]:
while not Ada.Text_IO.End_Of_File (Input) loop
   declare
      Line : String := Ada.Text_IO.Get_Line (Input);
   begin
      Count := Count + Boolean'Pos (Match (Line));
   end;
end loop;

Ada.Text_IO.Put_Line (Count'Image);

 260


----
[Back to Table of Contents](https://github.com/reznikmm/ada-howto/tree/advent-2020) 